# 附錄 -- 下載資料

In [1]:
import pandas as pd
import numpy as np
import requests as rq
import zipfile as zp
import os
import io
import pathlib

---
## Youbike 相關

### Youbike OD 資料

- 時間範圍： 2021 到 2023， 01 到 12 月，所有起訖資料
- 資料格式
    - json
        - 所有月份資料 .zip 網址
            - 該月份起迄資料 .csv

政府開放資料網站下載到的資料為 json 格式，裡面有個個月份的資料網址，格式為 zip

In [5]:

YB_OB_COLLECTION_URL = "https://quality.data.gov.tw/dq_download_json.php?nid=150635&md5_url=e51f13f6f3f14df40f17c175161be3cc"

all_youbike_od_collection_list:list[dict] = \
    rq.get(YB_OB_COLLECTION_URL).json()


In [12]:
[d['fileURL'] for d in all_youbike_od_collection_list]

['https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-02/202302_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2023/2023-01/202201_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-12/202212_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-11/202211_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-10/202210_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-09/202209_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-08/202208_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike_second_ticket_opendata/2022/2022-07/202207_YouBike2.0票證刷卡資料.zip',
 'https://tcgbusfs.blob.

In [2]:
youbike_data_path = pathlib.Path('../DATA') / "Youbike"

In [23]:
all_youbike_csv_files = [f for f in youbike_data_path.glob("*.csv")]
youbike_OD_combined = pd.concat(
    [pd.read_csv(f) for f in all_youbike_csv_files]
)


In [28]:
youbike_OD_combined.to_parquet(youbike_data_path / "yb.pq")

In [63]:
all_df = pd.read_parquet(youbike_data_path / "yb.pq")

In [65]:
#remove na
all_df = all_df.dropna()

In [61]:
for name in all_df.sort_values('rent_station')['rent_station'].unique():
    print(name)

六福公園
崇仰公園(公舘路255巷)
東新國小
瑠公公園
糖廍文化園區
nan


In [55]:
have_special_char = all_df.rent_station.str.contains(r'[\.\?]', na=True)

for name in all_df[have_special_char]['rent_station'].unique():
    print(name)

In [75]:
strang_station_name_map = {
"糖?文化園區": "糖廍文化園區",
"?公公園": "瑠公公園",
"東新國小.": "東新國小",
"六福公園.": "六福公園",
"崇仰公園(公?路255巷)": "崇仰公園(公舘路255巷)",
"瓦?溝(福真里)" : "瓦磘溝(福真里)",
"瓦?截流站" : "瓦磘截流站",
"?寮公園" : "獇寮公園",
}

In [76]:
all_df.rent_station = all_df.rent_station.replace(strang_station_name_map)
all_df.return_station = all_df.return_station.replace(strang_station_name_map)

/var/folders/jn/cpxfd61d5q51pz8dtwjztd0m0000gn/T/ipykernel_3536/2664486848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df.rent_station = all_df.rent_station.replace(strang_station_name_map)
/var/folders/jn/cpxfd61d5q51pz8dtwjztd0m0000gn/T/ipykernel_3536/2664486848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df.return_station = all_df.return_station.replace(strang_station_name_map)


In [77]:

have_special_char = (all_df.rent_station.str.contains(r'[\.\?]', na=True)) | (all_df.return_station.str.contains(r'[\.\?]', na=True))

for name in all_df[have_special_char]['rent_station'].unique():
    print(name)

for name in all_df[have_special_char]['return_station'].unique():
    print(name)

In [78]:
all_df[have_special_char]

,rent_time,rent_station,return_time,return_station,rent,infodate


In [79]:
all_df.to_parquet(youbike_data_path / "yb.pq")

### Youbike 座標以及可用車位

In [2]:
YB_STATION_INFO_URL = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"

yb_station_info = rq.get(YB_STATION_INFO_URL).json()

In [10]:
yb_station_df = pd.DataFrame(yb_station_info)
yb_station_df.sna = yb_station_df.sna.str.replace("YouBike2.0_", '')

/var/folders/jn/cpxfd61d5q51pz8dtwjztd0m0000gn/T/ipykernel_4892/4263609898.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  yb_station_df.sna = yb_station_df.sna.str.replace("YouBike2.0_", '')


In [11]:
yb_station_df

,sno,sna,tot,sbi,sarea,mday,lat,lng,ar,sareaen,snaen,aren,bemp,act,srcUpdateTime,updateTime,infoTime,infoDate
0,500101001,捷運科技大樓站,28,1,大安區,2023-09-03 17:31:05,25.02605,121.54360,復興南路二段235號前,Daan Dist.,YouBike2.0_MRT Technology Bldg. Sta.,No.235， Sec. 2， Fuxing S. Rd.,27,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:05,2023-09-03
1,500101002,復興南路二段273號前,21,5,大安區,2023-09-03 17:31:05,25.02565,121.54357,復興南路二段273號西側,Daan Dist.,YouBike2.0_No.273， Sec. 2， Fuxing S. Rd.,No.273， Sec. 2， Fuxing S. Rd. (West),16,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:05,2023-09-03
2,500101003,國北教大實小東側門,16,2,大安區,2023-09-03 17:31:05,25.02429,121.54124,和平東路二段96巷7號,Daan Dist.,YouBike2.0_NTUE Experiment Elementary School (...,No. 7， Ln. 96， Sec. 2， Heping E. Rd,14,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:05,2023-09-03
3,500101004,和平公園東側,11,11,大安區,2023-09-03 17:31:05,25.02351,121.54282,和平東路二段118巷33號,Daan Dist.,YouBike2.0_Heping Park (East),No. 33， Ln. 118， Sec. 2， Heping E. Rd,0,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:05,2023-09-03
4,500101005,辛亥復興路口西北側,16,16,大安區,2023-09-03 17:31:05,25.02153,121.54299,復興南路二段368號,Daan Dist.,YouBike2.0_Xinhai Fuxing Rd. Intersection (Nor...,No. 368， Sec. 2， Fuxing S. Rd.,0,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:05,2023-09-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,500119087,臺大總圖書館西南側,30,23,臺大公館校區,2023-09-03 17:31:14,25.01690,121.54031,臺大圖書館西南側,NTU Dist,YouBike2.0_NTU Main Library(Southwest),NTU Main Library(Southwest),7,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:14,2023-09-03
1305,500119088,臺大黑森林西側,20,5,臺大公館校區,2023-09-03 17:31:14,25.01995,121.54347,臺大霖澤館南側,NTU Dist,YouBike2.0_NTU Black Forest(West),NTU Tsai Lecture Hall(South),15,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:14,2023-09-03
1306,500119089,臺大獸醫館南側,24,5,臺大公館校區,2023-09-03 17:31:14,25.01791,121.54242,臺大獸醫系館南側,NTU Dist,YouBike2.0_NTU Dept. of Veterinary Medicine(So...,NTU Dept. of Veterinary Medicine(South),19,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:14,2023-09-03
1307,500119090,臺大新體育館東南側,40,19,臺大公館校區,2023-09-03 17:31:14,25.02112,121.53591,臺大體育館東側,NTU Dist,YouBike2.0_NTU Sports Center(Southeast),NTU Sports Center(East),21,1,2023-09-03 17:31:39,2023-09-03 17:31:51,2023-09-03 17:31:14,2023-09-03


In [71]:
yb_station_df.to_csv("../DATA/Youbike/TempYoubikeInfo.csv")

看看有多少站，在 OD 但不在 info裏面

In [74]:
yb_all_od = pd.read_parquet("../DATA/Youbike/yb.pq")
stations_in_OD = set(yb_all_od['rent_station'])
stations_in_info = set(yb_station_df['sna'])

In [75]:
no_info_stations = stations_in_OD - stations_in_info
no_info_stations

{'3樓客服中心',
 'YB_GTS測試',
 '一號船渠景觀橋',
 '七賢洛陽街口',
 '七賢自強路口東北側',
 '三民公園南側',
 '三民國小',
 '中國醫藥大學附設醫院臺北分�',
 '中山五福路口西北側',
 '中山青年路口西北側',
 '中林測試站',
 '中正和平路口西北側',
 '中正瑞源路口西南側',
 '中正高工(汕頭街口)',
 '中興中正路口',
 '中華復興路口(西南側)',
 '中華興中路口(東南角)',
 '信義服務中心',
 '僑安地下停車場(2號出口)東南�',
 '元氣大鎮社區',
 '凹子底森林公園(明誠三路側)',
 '凹子底森林公園(龍德路側)',
 '博愛重立路口南側',
 '台北市政府',
 '台北流行音樂中心(南港路)',
 '台北流行音樂中心(市民大道)',
 '台北醫學大學',
 '台鋁廣場',
 '和興街26巷口',
 '四維行政中心(四維三路)',
 '堯山延慶街口',
 '大裕昌富街口',
 '天母國中',
 '天母東路8巷/忠誠路二段154巷�',
 '安康路34巷24弄',
 '崇光女中',
 '延平北平路口',
 '建國同愛街口西北側',
 '捷運中央公園站(3號出口)',
 '捷運信義國小站(5號出口)',
 '捷運凹子底站(4號出口)',
 '捷運北投站(1號出口)(北投路側',
 '捷運台北小巨蛋站(2號出口)',
 '捷運善導寺站(3號出口)(忠孝東',
 '捷運善導寺站(3號出口)(忠孝路',
 '捷運市議會站(2號出口)',
 '捷運後驛站(1號出口北側)',
 '捷運後驛站(3號出口)',
 '捷運技擊館站(1號出口)',
 '捷運獅甲站(4號出口)',
 '捷運生態園區站(2號出口)',
 '捷運科技大樓站(台北教育大學',
 '捷運美麗島站(10號出口)',
 '捷運美麗島站(2號出口)',
 '捷運美麗島站(5號出口)',
 '捷運萬芳醫院站(興隆路三段115',
 '捷運鳳山站(1號出口)',
 '捷運鹽埕埔站(1號出口)',
 '文山兒童四公園',
 '新光自強路口',
 '新生南路一段113號',
 '新生高架停車場(林森北路107巷',
 '新興公園(昆陽街60巷)',
 '新興高中',
 '星雲金湖街口',
 '時代大道中華路口(西北側)',
 '板橋真武廟',


... 清一下吧@@

- 台北 -> 臺北
- 捷運台北小巨蛋 -> 捷運小巨蛋
- 有`(` 但沒有`)`
- 中國醫藥大學附設醫院臺北分� -> 中國醫藥大學附設醫院臺北分院
- 僑安地下停車場(2號出口)東南� -> 僑安地下停車場(2號出口)東南側
-  '捷運善導寺站(3號出口)(忠孝東',
- '捷運善導寺站(3號出口)(忠孝路',

先找最近的

In [51]:
import difflib

def get_closest_station(what_station):
    try:
        return difflib.get_close_matches(what_station, stations_in_info, 1, 0.6)[0]
    except:
        return None


best_match = {what_station: get_closest_station(what_station) for what_station in no_info_stations }


In [52]:
best_match

{'中林測試站': None,
 '羅斯福路三段333巷9號旁': '羅斯福路三段311號前',
 '獅甲社區(興仁國中棒球場)': None,
 '捷運台北小巨蛋站(2號出口)': '捷運小巨蛋站(2號出口)',
 '臺大卓越維修中心': None,
 '科工館車站': None,
 '新生高架停車場(林森北路107巷': '新生高架停車場(林森北路107巷口)',
 '蘆洲維修中心': None,
 '僑安地下停車場(2號出口)東南�': '僑安地下停車場(2號出口)東南側',
 '時代大道中華路口(西北側)': None,
 '石牌區民活動中心': '黎忠區民活動中心',
 '臺北市立大學(忠誠路二段207巷': '臺北市立大學(忠誠路二段207巷)',
 '捷運鹽埕埔站(1號出口)': '捷運關渡站(1號出口)',
 '中華復興路口(西南側)': '市民復興路口(東南側)',
 '台北流行音樂中心(南港路)': '臺北流行音樂中心(南港路)',
 '中正和平路口西北側': '建國和平路口西北側',
 '天母東路8巷/忠誠路二段154巷�': '天母東路8巷/忠誠路二段154巷口',
 '捷運萬芳醫院站(興隆路三段115': '捷運萬芳醫院站(興隆路三段115巷)',
 '捷運善導寺站(3號出口)(忠孝路': '捷運善導寺站(3號出口)(忠孝東路側)',
 '高雄女中': None,
 '高雄國際會議中心': None,
 '河東路建國三路口': '和平東建國南路口',
 '三民公園南側': '三民公園',
 '捷運信義國小站(5號出口)': '捷運小巨蛋站(5號出口)',
 '台北流行音樂中心(市民大道)': '臺北流行音樂中心(市民大道)',
 '台北醫學大學': '臺北醫學大學',
 '星雲金湖街口': '金湖星雲街口',
 '裕誠富民路口東北側': '民權光復路口(東北側)',
 '和興街26巷口': '和興路26巷口',
 '臺北魚市': '臺北花市',
 '高雄科技大學東北側': None,
 '濱海臨海路口(西南側)': '新生民族路口(西南側)',
 '捷運後驛站(3號出口)': '捷運後山埤站(3號出口)',
 '捷運美麗島站(10號出口)': '捷運關渡站(1號出口)',
 '銘傳大學(中山北路五段280巷口': '銘傳大學(中山北路五段2

先輸出，人工編輯

In [53]:
import yaml

with open('../DATA/Youbike/temp_change_name.yaml', 'w') as f:
    yaml.dump(best_match, f, encoding='utf-8', allow_unicode=True)

In [20]:
yb_all_od[yb_all_od.rent_station.isin(no_info_stations)]

,rent_time,rent_station,return_time,return_station,rent,infodate
49,2021-07-30 09:00:00,僑安地下停車場(2號出口)東南�,2021-07-30 09:00:00,信義延吉街口,00:04:48,2021-07-30
209,2021-07-30 18:00:00,羅斯福路三段333巷9號旁,2021-07-30 18:00:00,臺大椰林小舖,00:04:19,2021-07-30
238,2021-07-30 01:00:00,僑安地下停車場(2號出口)東南�,2021-07-30 05:00:00,景勤二號公園,04:22:58,2021-07-30
479,2021-07-07 18:00:00,僑安地下停車場(2號出口)東南�,2021-07-07 18:00:00,臨江街夜市(通化街101巷口),00:13:00,2021-07-07
487,2021-07-07 18:00:00,辛亥基隆路口,2021-07-07 18:00:00,和平臥龍街口,00:10:39,2021-07-07
...,...,...,...,...,...,...
250999,2021-04-21 12:00:00,臺大仰萃樓東南側,2021-04-21 12:00:00,臺大總圖書館西南側,00:06:03,2021-04-21
251006,2021-04-21 12:00:00,羅斯福路三段333巷9號旁,2021-04-21 12:00:00,臺大鹿鳴堂東側,00:05:44,2021-04-21
251011,2021-04-21 12:00:00,臺大仰萃樓東南側,2021-04-21 12:00:00,臺大樂學館東側,00:06:08,2021-04-21
251019,2021-04-21 12:00:00,臺大仰萃樓東南側,2021-04-21 12:00:00,臺大管理學院二館北側,00:09:30,2021-04-21


In [65]:
query_text = "辛亥路五段73巷口"
yb_all_od[yb_all_od.rent_station.isin(no_info_stations)].query(f"rent_station.str.contains('{query_text}')")#.query("rent_time < '2021-07-31'")

,rent_time,rent_station,return_time,return_station,rent,infodate
128419,2022-10-23 06:00:00,辛亥路五段73巷口,2022-10-23 06:00:00,捷運辛亥站,00:31:12,2022-10-23
128420,2022-10-23 07:00:00,辛亥路五段73巷口,2022-10-23 08:00:00,捷運大坪林站(1號出口),00:17:03,2022-10-23
128421,2022-10-23 19:00:00,辛亥路五段73巷口,2022-10-23 19:00:00,捷運辛亥站,00:02:42,2022-10-23
128422,2022-10-23 21:00:00,辛亥路五段73巷口,2022-10-23 21:00:00,辛亥路五段73巷口,00:06:52,2022-10-23
128423,2022-10-23 22:00:00,辛亥路五段73巷口,2022-10-23 23:00:00,木柵光輝路口,00:06:23,2022-10-23
...,...,...,...,...,...,...
2576872,2023-01-19 06:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:04:55,2023-01-19
2576873,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:04:57,2023-01-19
2576874,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運萬芳醫院站(興隆路三段115,00:05:09,2023-01-19
2576875,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:06:54,2023-01-19


有一些站點被移除，但其實很直得探討。我將手動增加資訊
- 辛亥路五段73巷口
- 機場捷運台北車站(6號出口)

In [78]:
import yaml
with open('../DATA/Youbike/temp_change_name＿changed.yaml', 'r') as f:
    manual_mapping_stations = yaml.safe_load(f)

In [79]:
manual_mapping_stations

{'中國醫藥大學附設醫院臺北分�': '中國醫藥大學附設醫院臺北分院',
 '僑安地下停車場(2號出口)東南�': '僑安地下停車場(2號出口)東南側',
 '台北市政府': '臺北市政府',
 '台北流行音樂中心(南港路)': '臺北流行音樂中心(南港路)',
 '台北流行音樂中心(市民大道)': '臺北流行音樂中心(市民大道)',
 '台北醫學大學': '臺北醫學大學',
 '和興街26巷口': '和興路26巷口',
 '天母東路8巷/忠誠路二段154巷�': '天母東路8巷/忠誠路二段154巷口',
 '安康路34巷24弄': '安康路32巷24弄',
 '捷運北投站(1號出口)(北投路側': '捷運北投站(1號出口)(北投路側)',
 '捷運台北小巨蛋站(2號出口)': '捷運小巨蛋站(2號出口)',
 '捷運善導寺站(3號出口)(忠孝東': '捷運善導寺站(3號出口)(忠孝東路側)',
 '捷運善導寺站(3號出口)(忠孝路': '捷運善導寺站(3號出口)(忠孝東路側)',
 '捷運科技大樓站(台北教育大學': '捷運科技大樓站(台北教育大學)',
 '捷運萬芳醫院站(興隆路三段115': '捷運萬芳醫院站(興隆路三段115巷)',
 '新生高架停車場(林森北路107巷': '新生高架停車場(林森北路107巷口)',
 '新興公園(昆陽街60巷)': '新新公園(昆陽街60巷)',
 '星雲金湖街口': '金湖星雲街口',
 '民善新湖二路口(家樂福內湖店': '民善新湖二路口(家樂福內湖店)',
 '民權東龍江路口': '民權龍江路口',
 '環南市場': '環南綜合市場',
 '臺北市立大學(忠誠路二段207巷': '臺北市立大學(忠誠路二段207巷)',
 '臺大仰萃樓東南側': '臺大禮賢樓東南側',
 '萬華國中': '萬華國中_1',
 '辛亥基隆路口': '臺大土木研究大樓前',
 '銘傳大學(中山北路五段280巷口': '銘傳大學(中山北路五段280巷口)'}

In [80]:
yb_all_od.rent_station = yb_all_od.rent_station.replace(manual_mapping_stations)
yb_all_od.return_station = yb_all_od.return_station.replace(manual_mapping_stations)

In [81]:
query_text = "辛亥路五段73巷口"
yb_all_od[yb_all_od.rent_station.isin(no_info_stations)].query(f"rent_station.str.contains('{query_text}')")#.query("rent_time < '2021-07-31'")

,rent_time,rent_station,return_time,return_station,rent,infodate
128419,2022-10-23 06:00:00,辛亥路五段73巷口,2022-10-23 06:00:00,捷運辛亥站,00:31:12,2022-10-23
128420,2022-10-23 07:00:00,辛亥路五段73巷口,2022-10-23 08:00:00,捷運大坪林站(1號出口),00:17:03,2022-10-23
128421,2022-10-23 19:00:00,辛亥路五段73巷口,2022-10-23 19:00:00,捷運辛亥站,00:02:42,2022-10-23
128422,2022-10-23 21:00:00,辛亥路五段73巷口,2022-10-23 21:00:00,辛亥路五段73巷口,00:06:52,2022-10-23
128423,2022-10-23 22:00:00,辛亥路五段73巷口,2022-10-23 23:00:00,木柵光輝路口,00:06:23,2022-10-23
...,...,...,...,...,...,...
2576872,2023-01-19 06:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:04:55,2023-01-19
2576873,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:04:57,2023-01-19
2576874,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運萬芳醫院站(興隆路三段115巷),00:05:09,2023-01-19
2576875,2023-01-19 07:00:00,辛亥路五段73巷口,2023-01-19 07:00:00,捷運辛亥站,00:06:54,2023-01-19


## 手動增加一些已移除之站點後，重新 merge

In [85]:
yb_station_info = (pd.read_csv('../DATA/Youbike/TempYoubikeInfo.csv')[["sna", "tot", "lat", "lng", 'ar']]
                   .rename(columns = {
                       "sna":"name", 
                       "tot": "total",
                       "ar": "address"}))

In [86]:
yb_station_info

,name,total,lat,lng,address
0,捷運科技大樓站,28,25.02605,121.54360,復興南路二段235號前
1,復興南路二段273號前,21,25.02565,121.54357,復興南路二段273號西側
2,國北教大實小東側門,16,25.02429,121.54124,和平東路二段96巷7號
3,和平公園東側,11,25.02351,121.54282,和平東路二段118巷33號
4,辛亥復興路口西北側,16,25.02153,121.54299,復興南路二段368號
...,...,...,...,...,...
1306,臺大獸醫館南側,24,25.01791,121.54242,臺大獸醫系館南側
1307,臺大新體育館東南側,40,25.02112,121.53591,臺大體育館東側
1308,臺大明達館北側(員工宿舍),18,25.01816,121.54469,明達館北側前空地
1309,辛亥路五段73巷口,23,24.99818,121.55312,已移除。手動增加資訊


In [88]:
yb_station_info.to_parquet("../DATA/Youbike/yb_info.pq")

注意！先不急著把站點不存在的OD刪掉，因為可能起沒有在台北，但迄在台北。這樣會低估還車使用率